In [118]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV

In [98]:
df = pd.read_csv(r"C:\Users\SUMIT MITRA\Churn_ Data.csv")

In [99]:
target_column = 'target'  
y = df[target_column]
X = df.drop(columns=[target_column])

In [100]:
df = pd.concat([X, y], axis=1)
df = df.drop_duplicates()
# Separate again
X = df.drop(columns=[target_column])
y = df[target_column]

In [101]:
for col in X.columns:
    if X[col].nunique() == 1:
        X = X.drop(col, axis=1)

In [102]:
selector = VarianceThreshold(threshold=0.8)
X = pd.DataFrame(selector.fit_transform(X), columns=X.columns[selector.get_support(indices=True)])

In [103]:
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
X = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]
y = y.loc[X.index]  

In [104]:
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] >= 0.9)]
X = X.drop(columns=to_drop)

In [117]:
def calculate_vif(X):
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif.dropna()  # Drop rows with NaN values
vif_df = calculate_vif(X)
while vif_df['VIF'].max() > 5:
    max_vif_col = vif_df.loc[vif_df['VIF'].idxmax(), 'Variable']
    X = X.drop(columns=[max_vif_col])
    vif_df = calculate_vif(X)

C:\Users\SUMIT MITRA\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


In [106]:
df_processed = pd.concat([X.reset_index(drop=True), y.reset_index(drop=True)], axis=1)
df_processed.to_csv('processed_data.csv', index=False)

In [107]:
# Splitting the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logistic_model = LogisticRegression(solver='liblinear', max_iter=1000)
# Training the logistic regression model
logistic_model.fit(X_train, y_train)
# Predicting on the test set
y_pred = logistic_model.predict(X_test)
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8125


In [119]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'feature_selection__n_features_to_select': [None, 5, 10], 
    'logistic_regression__C': [0.1, 1, 10], 
    'logistic_regression__solver': ['liblinear', 'lbfgs']  
}
# Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8152985074626866
